In [1]:
import cv2, numpy as np, pandas as pd
from pathlib import Path
from tqdm.auto import tqdm


ORIG_DIR = Path("./dataset/autos")        # donde están auto001.jpeg, ...
PROC_DIR = Path("./dataset/autos_proc")   # guardaremos las imágenes B/N
BITS_DIR = Path("./dataset/autos_bits")   # guardaremos .npy con los bits
PROC_DIR.mkdir(parents=True, exist_ok=True)
BITS_DIR.mkdir(parents=True, exist_ok=True)

/home/jpalaciosch/Documents/UNAL/Septimo semestre/Comunicaciones/Proyecto final/venv/lib64/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
